## Testing on Data
Opening the database files


~corpus folder

In [2]:
from data_loc import directory_path

data_path = directory_path

In [1]:
import json

with open("wamex/data/corpus/document2anumber.json", 'r') as file:
    data = json.load(file)

with open("wamex/data/corpus/document2keywords.json", 'r') as file:
    datafd = json.load(file)

with open("wamex/data/corpus/keyword2documents.json", 'r') as file:
    datakeys = json.load(file)

# data = json.load(f)
# datafd = json.load(fd)
# datakeys = json.load(fkeys)

# f.close()
# fd.close()
# fkeys.close()

for i, d in enumerate(data):
    if i > 0:
        break
    print("document2anumber.json:")
    print(d, data[d])

print()

for i, d in enumerate(datafd):
    if i > 0:
        break
    print("document2keywords.json:")
    print(d, datafd[d])

print()

for i, d in enumerate(datakeys):
    if i > 0:
        break
    print("keyword2documents.json:")
    print(d, datakeys[d])

print()

print("~~~~~~'a073533_nf_e09_1004_2006a_12145691.pdf' testing:")
print(data['a073533_nf_e09_1004_2006a_12145691.pdf'])
print(datafd['a073533_nf_e09_1004_2006a_12145691.pdf'])

print()


with open("wamex/data/wamex_metadata/073533.json", 'r') as file:
    meta_data = json.load(file)

print("073533.json testing:")
print(meta_data)



FileNotFoundError: [Errno 2] No such file or directory: 'wamex/data/corpus/document2anumber.json'

### Opening PDF 

In [ ]:
# ! pip install pypdf
# https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
from pypdf import PdfReader 

reader = PdfReader("wamex/data/wamex_pdf/a073533_nf_e09_1004_2006a_12145691.pdf")

print(len(reader.pages))

# page = reader.pages[0]
# print(page.extract_text())

# page1 = reader.pages[1]
# print(page1.extract_text())

# page5 = reader.pages[5]
# print(page5.extract_text())

for i in range(len(reader.pages)):
    if i > 1:
        print(f"Page {i+1}:")
        print(reader.pages[i].extract_text())

23
Page 3:
Midwest Corporation Limited 
Annual Report E09/1004 September 2006  
                                         Page 1 of 21 
September 2006 
 
 
 
 
EXECUTIVE SUMMARY  
 
This report has been prepared in accordance with the requirements of the Department of Industry and 
Resources (DOIR) and details exploration activities conducted by Midwest Corporation Limited (Midwest) 
over its New Forest Project (Mt Aubrey tenement E09/1004) in the Murchison Region of Western 
Australia, 130km north of Mullewa and 520km north of Perth.  The reporting period is from the 26 
September 2005 to the 25 September 2006. 
 
No ground disturbing fieldwork was undertaken within the tenement during the reporting period since 
Midwest was only successful in obtaining a signed Heritage Agreement from the Yamatji Land and Sea 
Council part the way through the reporting period. Additionally, a series of cyclonic events in the 
northwest of Western Australia, left large areas adjacent to the Murchison R